In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df1 = pd.read_csv('users.csv')
df2 = pd.read_csv('google_users.csv')

In [3]:
#check df1
df1.head()

,user_id,is_activated,signed_up_on,last_login,sign_up_source,unsubscribed,user_type
0,48063,True,13/06/2019 23:37,13/06/2019 23:39,NaN,0,1
1,50616,True,5/07/2019 10:14,5/07/2019 10:14,google,0,1
2,46065,True,4/06/2019 7:26,4/06/2019 7:26,google,0,1
3,52503,True,7/07/2019 0:29,7/07/2019 2:53,NaN,0,1
4,55278,True,7/07/2019 7:22,7/07/2019 7:27,NaN,0,1


In [4]:
#check df2
df2.head()

,id,Unnamed: 1,user_id,browser_ language_code,created_on,device_cat
0,7659.0,NaN,41625.0,en,7/05/2019 12:58,Tablet
1,7770.0,NaN,41736.0,en-GB,12/05/2019 16:42,Desktop
2,7881.0,NaN,41958.0,en,13/05/2019 15:26,Desktop
3,7992.0,NaN,42513.0,en,23/05/2019 15:15,Mobile
4,8103.0,NaN,42624.0,en,25/05/2019 3:31,Desktop


In [5]:
#join df2 to df1 to get extra google info
df3 = df1.merge(df2, how='left')
df3.head()


,user_id,is_activated,signed_up_on,last_login,sign_up_source,unsubscribed,user_type,id,Unnamed: 1,browser_ language_code,created_on,device_cat
0,48063,True,13/06/2019 23:37,13/06/2019 23:39,NaN,0,1,NaN,NaN,NaN,NaN,NaN
1,50616,True,5/07/2019 10:14,5/07/2019 10:14,google,0,1,9990.0,NaN,en,5/07/2019 10:14,Tablet
2,46065,True,4/06/2019 7:26,4/06/2019 7:26,google,0,1,8658.0,NaN,en-GB,4/06/2019 7:26,Desktop
3,52503,True,7/07/2019 0:29,7/07/2019 2:53,NaN,0,1,11322.0,NaN,en,7/07/2019 2:53,Desktop
4,55278,True,7/07/2019 7:22,7/07/2019 7:27,NaN,0,1,NaN,NaN,NaN,NaN,NaN


In [6]:
#find sign-up number in July 2019
df3 = df3.assign(signed_date = pd.to_datetime(df3['signed_up_on'],format='%d/%m/%Y %H:%M'))
july_2019 = (df3['signed_date'] > '2019-6-30') & (df3['signed_date'] < '2019-8-1')
df3_q1 = df3.loc[july_2019].sort_values(by='signed_date', ascending= True)
#q1 ans
print ("Q1 ans =",len(df3_q1))

Q1 ans = 454


In [7]:
#what lifetime value of users that signed up via email during june, while $500 for type0 and $10 for type1
df3_q2 = df3[(df3['signed_date']>'2019-5-31') & (df3['signed_date']<'2019-7-1')]
df3_q2['sign_up_source'] = df3_q2['sign_up_source'].fillna('email')
df3_q2 = df3_q2.query('sign_up_source == "email"')
df3_q2 = df3_q2.assign(lifetime = 0)
df3_q2['lifetime'].loc[df3_q2['user_type'] == 1] = 10; df3_q2['lifetime'].loc[df3_q2['user_type'] == 0] = 500
df3_q2_1 = df3_q2['lifetime'].sum()
#if exclude unsubscribed
df3_q2 = df3_q2.query('unsubscribed == 0')
df3_q2_2 = df3_q2['lifetime'].sum()
#q2 ans
print ("Q2 ans =",df3_q2_1,", if exclude unsubscribed",df3_q2_2)

Q2 ans = 7700 , if exclude unsubscribed 7690


In [8]:
#my manager live in Brisbane, Australia, need to change timezone from UTC to Brisbane's timezone and find which day is the most signup from LinkedIn
#Assign Time Zone
df3['signed_date'] = pd.DatetimeIndex(df3['signed_date']).tz_localize('UTC')
#Convert Time Zone to new Column
df3 = df3.assign(signed_date_in_aus = pd.DatetimeIndex(df3['signed_date']).tz_convert('Australia/Brisbane'))
#Filter only LinkedIn signed up
df3_q3 = df3.query('sign_up_source == "linkedin"')
#Convert datetime to date
df3_q3['signed_date_in_aus'] = df3_q3['signed_date_in_aus'].dt.date
#Group by and sizing row
df3_q3_1 = pd.DataFrame(df3_q3.groupby(['signed_date_in_aus','sign_up_source']).size())
#rename aggregate to count and sort
df3_q3_1 = df3_q3_1.reset_index().rename(columns ={0:"Count"}).sort_values(by='Count',ascending=False)
#q3 ans
print ("Q3 ans =",df3_q3_1.head(2))


Q3 ans =    signed_date_in_aus sign_up_source  Count
16         2019-07-27       linkedin      6
6          2019-07-08       linkedin      6
